In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pooch
from IPython.display import display, HTML
import plotly.express as px
from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties
from datetime import datetime


/Users/susierwu/miniconda3/envs/ar6/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
sys.path.append('../') 
from utils.majorghg_modA import *
from utils.majorghg_modB_whRF import * 

# A. first run FAIR model, as output parameters of fair will be used as input for calculating d-GWP

## 1. Create FaIR instance

To reproduce an AR6-like run, we want to allow methane lifetime to be affected by all its relevant chemical precursors (NOx, VOCs, etc) so we set the `ch4_method` flag to `Thornhill2021` (see https://docs.fairmodel.net/en/latest/api_reference.html#fair.FAIR for all of the options for initialising `FAIR`).

In [3]:
f = FAIR(ch4_method="Thornhill2021")

## 2. Define time horizon

A lot of analysis uses 2100 as the time horizon, but 2300 is an interesting end point to see the effects of long-term climate change. We'll set 2300 as the last time bound, so the last emissions time point is 2299.5. We could even run to 2500, as the scenarios are defined that far.

In [4]:
f.define_time(2000, 2500, 1)  # start=1750(baselinE), end=2300, step

## 3. Define scenarios

Since the eight tier 1 & tier 2 SSPs are shipped with RCMIP, and they are quite familiar, we'll use these scenarios. We'll use the `fill_from_rcmip()` function from FaIR, so these have to use the same scenario names that appear in the RCMIP database.

In [5]:
scenarios = ["ssp119", "ssp126", "ssp245",  "ssp585"]
#scenarios = ["ssp119", "ssp126", "ssp245", "ssp370", "ssp434", "ssp460", "ssp534-over", "ssp585"]

In [6]:
f.define_scenarios(scenarios)

## 4. Define configs

In [7]:
fair_params_1_2_0_obj = pooch.retrieve(
    url = 'https://zenodo.org/record/8399112/files/calibrated_constrained_parameters.csv',
    known_hash = 'md5:de3b83432b9d071efdd1427ad31e9076',
)

In [8]:
df_configs = pd.read_csv(fair_params_1_2_0_obj, index_col=0)
configs = df_configs.index  # this is used as a label for the "config" axis
f.define_configs(configs)

In [9]:
#df_configs.head()

## 5. Define species and properties

We will use FaIR's default list of 63 species. They are often run with default properties that are included in the model code. However, as part of the v1.1 calibration, some defaults are modified, such as the sensitivity of chemical precursors to methane lifetime. Rather than manually overriding this by setting `species_configs`, it is cleaner to modify the defaults in the CSV file that is read in to define the `species` and `properties`. 

In fact, as this only reads in and defines `species` and `properties` (not `species_configs`), the default (no `filename`) argument could be used here, but it is good practice in my opinion to put species, properties and configs in the same file, and to use the same file to read in `properties` and `species_configs`.

If you're following along at home, feel free to insert a new cell after this one and inspect what the `species` and `properties` actually are.

In [10]:
#https://github.com/OMS-NetZero/FAIR/blob/master/examples/data/species_configs_properties_calibration1.2.0.csv 
#saved to 'data/ ... 
species, properties = read_properties(filename='../data/species_configs_properties_calibration1.2.0.csv')
f.define_species(species, properties)

In [11]:
#properties

## 6. Modify run options

Not necessary, as we made all of our choices on initialisation (step 1).

## 7. Create input and output xarrays

If this runs without error, the problem is consistently and completely set up: we then just need to add data.

In [12]:
f.allocate()

## 8. Fill in data

### 8a. emissions, solar forcing, and volcanic forcing

We can use the convenience function `fill_from_rcmip()` to fill in the emissions. Remember that not all `species` are things that take emissions, so if you see NaNs below, don't panic.

In [13]:
f.fill_from_rcmip()

In [14]:
#f.emissions

There is an issue with the RCMIP NOx emissions; the units are different for biomass burning emissions (Tg NO/yr) to the other emissions from fossil fuels, industry and agriculture (Tg NO2/yr). v1.1 of the calibration uses the corrected NOx emissions expressed in Tg NO2/yr, so we also have to correct them in FaIR for consistency.

We download the RCMIP emissions file, and pull out the relevant sectors, update the unit, and finally override the correct entry of `f.emissions`.

Notes on the below:

- 46.006 is the molecular weight of NO2 (g/mol).
- 30.006 is the molecular weight of NO (g/mol).
- The final `[:550, None]` is to transfer the data coming in from RCMIP (dimension (750,), a timeseries of annual emissions) into the correct shape for our problem (550, 1001). Since we are looping over the `scenario` dimension and selecting it, and we are selecting NOx from the `species` dimension, these axes are collapsed and we're left with (`timepoints`, `configs`). The RCMIP data starts in 1750 as does our emissions data; if there is a mismatch in the start date, it would be necessary to select the correct slice from the RCMIP `DataFrame` that is loaded in. For a reminder of the dimensioning in FaIR 2.1, see https://docs.fairmodel.net/en/latest/intro.html#dimensionality.

In [15]:
rcmip_emissions_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-emissions-annual-means-v5-1-0.csv",
    known_hash="md5:4044106f55ca65b094670e7577eaf9b3",
)
df_emis = pd.read_csv(rcmip_emissions_file)
gfed_sectors = [
    "Emissions|NOx|MAGICC AFOLU|Agricultural Waste Burning",
    "Emissions|NOx|MAGICC AFOLU|Forest Burning",
    "Emissions|NOx|MAGICC AFOLU|Grassland Burning",
    "Emissions|NOx|MAGICC AFOLU|Peat Burning",
]
for scenario in scenarios:
    f.emissions.loc[dict(specie="NOx", scenario=scenario)] = (
        df_emis.loc[
            (df_emis["Scenario"] == scenario)
            & (df_emis["Region"] == "World")
            & (df_emis["Variable"].isin(gfed_sectors)),
            "2000":"2500",       #change back to 2300 as default
        ]
        .interpolate(axis=1)
        .values.squeeze()
        .sum(axis=0)
        * 46.006
        / 30.006
        + df_emis.loc[
            (df_emis["Scenario"] == scenario)
            & (df_emis["Region"] == "World")
            & (df_emis["Variable"] == "Emissions|NOx|MAGICC AFOLU|Agriculture"),
            "2000":"2500",
        ]
        .interpolate(axis=1)
        .values.squeeze()
        + df_emis.loc[
            (df_emis["Scenario"] == scenario)
            & (df_emis["Region"] == "World")
            & (df_emis["Variable"] == "Emissions|NOx|MAGICC Fossil and Industrial"),
            "2000":"2500",
        ]
        .interpolate(axis=1)
        .values.squeeze()
    )[:500, None]  # 550 if ending year 2300 and starting 1750, 

Now we fetch and fill in the solar and volcanic forcing. As these are forcing-driven time series, if we want to vary the uncertainties in the forcing, this has to happen before FaIR is run (see https://github.com/OMS-NetZero/FAIR/issues/126).

In [16]:
solar_obj = pooch.retrieve(
    url = 'https://raw.githubusercontent.com/chrisroadmap/fair-add-hfc/main/data/solar_erf_timebounds.csv',
    known_hash = 'md5:98f6f4c5309d848fea89803683441acf',
)

In [17]:
volcanic_obj = pooch.retrieve(
    url = 'https://raw.githubusercontent.com/chrisroadmap/fair-calibrate/main/data/forcing/volcanic_ERF_1750-2101_timebounds.csv',
    known_hash = 'md5:c0801f80f70195eb9567dbd70359219d',
)

#### <font color = "red">df_solar only has data 1750 - up tp 2300 </font>

In [18]:
df_solar = pd.read_csv(solar_obj, index_col="year")
df_volcanic = pd.read_csv(volcanic_obj)

Remembering that everything that is not emissions is on `timebounds`, there is always one more `timebounds` than `timepoints`, so we define arrays of length 551 (1750 to 2300, inclusive).

Volcanic forcing is given monthly, so we average the 12 previous months for each `timebounds` volcanic forcing.

Volcanic forcing here follows the CMIP6 ScenarioMIP convention of a 10 year ramp down to zero from the last year of data (here 2019). Again a little bit of ninja skill with indexing is needed.

In [19]:
l = 501 # since we change ending year to 2500, AND starting year to 2000 here we need new array.dim to (751,1001)

solar_forcing = np.zeros(l)
volcanic_forcing = np.zeros(l)
volcanic_forcing[:352] = df_volcanic.erf.values
solar_forcing[:301] = df_solar["erf"].loc[2000:2300].values

trend_shape = np.ones(l)
trend_shape[:271] = np.linspace(0, 1, 271) 

We then use our calibrated, constrained ensemble to individually scale the volcanic forcing time series, and the solar amplitude and trend:

In [20]:
fill(
    f.forcing,
    volcanic_forcing[:, None, None] * df_configs["fscale_Volcanic"].values.squeeze(),
    specie="Volcanic",
)
fill(
    f.forcing,
    solar_forcing[:, None, None] * df_configs["fscale_solar_amplitude"].values.squeeze()
    + trend_shape[:, None, None] * df_configs["fscale_solar_trend"].values.squeeze(),
    specie="Solar",
)

### 8b. Fill in climate_configs

This is relatively straightforward from the calibrated, constrained dataset.

In [21]:
fill(f.climate_configs["ocean_heat_capacity"], df_configs.loc[:, "clim_c1":"clim_c3"].values)
fill(
    f.climate_configs["ocean_heat_transfer"],
    df_configs.loc[:, "clim_kappa1":"clim_kappa3"].values,
)
fill(f.climate_configs["deep_ocean_efficacy"], df_configs["clim_epsilon"].values.squeeze())
fill(f.climate_configs["gamma_autocorrelation"], df_configs["clim_gamma"].values.squeeze())
fill(f.climate_configs["sigma_eta"], df_configs["clim_sigma_eta"].values.squeeze())
fill(f.climate_configs["sigma_xi"], df_configs["clim_sigma_xi"].values.squeeze())
fill(f.climate_configs["seed"], df_configs["seed"])
fill(f.climate_configs["stochastic_run"], True)
fill(f.climate_configs["use_seed"], True)
fill(f.climate_configs["forcing_4co2"], df_configs["clim_F_4xCO2"])

### 8c. Fill in species_configs

Firstly we want to get the defaults from our new species/properties/configs file

In [22]:
f.fill_species_configs(filename='../data/species_configs_properties_calibration1.2.0.csv')

Then, we overwrite the `species_configs` that are varies as part of the probablistic sampling. This makes heavy use of the `fill()` convenience function.

In [23]:
# carbon cycle
fill(f.species_configs["iirf_0"], df_configs["cc_r0"].values.squeeze(), specie="CO2")
fill(f.species_configs["iirf_airborne"], df_configs["cc_rA"].values.squeeze(), specie="CO2")
fill(f.species_configs["iirf_uptake"], df_configs["cc_rU"].values.squeeze(), specie="CO2")
fill(f.species_configs["iirf_temperature"], df_configs["cc_rT"].values.squeeze(), specie="CO2")

# aerosol indirect
fill(f.species_configs["aci_scale"], df_configs["aci_beta"].values.squeeze())
fill(f.species_configs["aci_shape"], df_configs["aci_shape_so2"].values.squeeze(), specie="Sulfur")
fill(f.species_configs["aci_shape"], df_configs["aci_shape_bc"].values.squeeze(), specie="BC")
fill(f.species_configs["aci_shape"], df_configs["aci_shape_oc"].values.squeeze(), specie="OC")

# aerosol direct
for specie in [
    "BC", 
    "CH4", 
    "N2O",
    "NH3", 
    "NOx",
    "OC", 
    "Sulfur", 
    "VOC",
    "Equivalent effective stratospheric chlorine"
]:
    fill(f.species_configs["erfari_radiative_efficiency"], df_configs[f"ari_{specie}"], specie=specie)

# forcing scaling
for specie in [
    "CO2", 
    "CH4", 
    "N2O", 
    "Stratospheric water vapour",
    "Contrails", 
    "Light absorbing particles on snow and ice", 
    "Land use"
]:
    fill(f.species_configs["forcing_scale"], df_configs[f"fscale_{specie}"].values.squeeze(), specie=specie)
# the halogenated gases all take the same scale factor
for specie in [
    "CFC-11",
    "CFC-12",
    "CFC-113",
    "CFC-114",
    "CFC-115",
    "HCFC-22",
    "HCFC-141b",
    "HCFC-142b",
    "CCl4",
    "CHCl3",
    "CH2Cl2",
    "CH3Cl",
    "CH3CCl3",
    "CH3Br",
    "Halon-1211",
    "Halon-1301",
    "Halon-2402",
    "CF4",
    "C2F6",
    "C3F8",
    "c-C4F8",
    "C4F10",
    "C5F12",
    "C6F14",
    "C7F16",
    "C8F18",
    "NF3",
    "SF6",
    "SO2F2",
    "HFC-125",
    "HFC-134a",
    "HFC-143a",
    "HFC-152a",
    "HFC-227ea",
    "HFC-23",
    "HFC-236fa",
    "HFC-245fa",
    "HFC-32",
    "HFC-365mfc",
    "HFC-4310mee",
]:
    fill(f.species_configs["forcing_scale"], df_configs["fscale_minorGHG"].values.squeeze(), specie=specie)

# ozone
for specie in ["CH4", "N2O", "Equivalent effective stratospheric chlorine", "CO", "VOC", "NOx"]:
    fill(f.species_configs["ozone_radiative_efficiency"], df_configs[f"o3_{specie}"], specie=specie)

# initial value of CO2 concentration (but not baseline for forcing calculations)
fill(
    f.species_configs["baseline_concentration"], 
    df_configs["cc_co2_concentration_1750"].values.squeeze(), 
    specie="CO2"
)

### 8d. Initial conditions

It's important these are defined, as they are NaN by default, and it's likely you'll run into problems.

In [24]:
#f.species_configs["baseline_concentration"]
#f.species_configs["baseline_concentration"]    #.loc()

In [25]:
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

## 9. Run

In [26]:
f.run()

Running 4004 projections in parallel: 100%|█| 500/500 [01:15<00:00,  6.59timeste


# B. Rolling-Y, set H_max = 0, get Y[0] output from fair each fun,  

### Q: if I have a middle-of-road SSP260 for my LCA (inventory stage), can I run FaIR ssp460 as the metrics for Impact Assessment? 
#### assuming regardless of Socio-pathway, RCP is the same under 60 so that we can use the ssp460 metrics for SSP260? 

In [27]:
ssp = ["ssp119", "ssp126", "ssp245",  "ssp585"]  #["ssp119", "ssp126", "ssp245", "ssp370", "ssp434", "ssp460", "ssp534-over", "ssp585"],
"""  rolling Y, each year is the new MY """
y_start = 2000 
H_m = 1
y_index = np.arange(25,25+100,1) # from 2025  

# to save alpha
alpha_ch4_l_allSSP , alpha_n2o_l_allSSP = [],[]
# to save ERF 
y0_co2_erf_l_allSSP, y0_ch4_erf_l_allSSP, y0_n2o_erf_l_allSSP = [],[],[]
# to save concentration 
y0_co2_C_l_allSSP, y0_ch4_C_l_allSSP, y0_n2o_C_l_allSSP = [],[],[]

for x in ssp: 
    alpha_ch4_l , alpha_n2o_l = [],[]
    y0_co2_erf_l, y0_ch4_erf_l, y0_n2o_erf_l = [],[],[]
    y0_co2_c_l, y0_ch4_c_l, y0_n2o_c_l = [],[],[]
    
    for y in y_index: 
        f36 = majorghg_get_f(f = f, H_max=H_m, ts_per_year = 1, year_index = y , scn = x) 
        alpha_ch4, alpha_n2o = f36.get_ch4_n2o_alpha()
        y0_co2_c, y0_ch4_c, y0_n2o_c = f36.get_majorghg_concentration()
        
        alpha_ch4_l.append(alpha_ch4) 
        alpha_n2o_l.append(alpha_n2o)
        # to get and save concentration 
        y0_co2_c_l.append(y0_co2_c)
        y0_ch4_c_l.append(y0_ch4_c)
        y0_n2o_c_l.append(y0_n2o_c)
        
        # to get and save ERF
        co2_erf_diff_t = f36.get_co2_1ppm_erf()
        ch4_erf_diff_t = f36.get_ch4_1ppb_erf()
        n2o_erf_diff_t  = f36.get_n2o_1ppb_erf()
        #co2_erf_diff_t[y] is the MY[Y[0]] output   
        y0_co2_erf, y0_ch4_erf, y0_n2o_erf = co2_erf_diff_t[y], ch4_erf_diff_t[y], n2o_erf_diff_t[y]
        y0_co2_erf_l.append(y0_co2_erf)
        y0_ch4_erf_l.append(y0_ch4_erf)
        y0_n2o_erf_l.append(y0_n2o_erf)
    
    alpha_ch4_l_allSSP.append(alpha_ch4_l) 
    alpha_n2o_l_allSSP.append(alpha_n2o_l)
    y0_co2_erf_l_allSSP.append(y0_co2_erf_l)
    y0_ch4_erf_l_allSSP.append(y0_ch4_erf_l)
    y0_n2o_erf_l_allSSP.append(y0_n2o_erf_l)
    y0_co2_C_l_allSSP.append(y0_co2_c_l)
    y0_ch4_C_l_allSSP.append(y0_ch4_c_l)
    y0_n2o_C_l_allSSP.append(y0_n2o_c_l)
        

#### for y0_co2_erf_l_allSSP, first index for the list for SSP[x], 2nd index is for MY[0-99]

In [28]:
print(y0_co2_erf_l_allSSP[0][99], y0_co2_erf_l_allSSP[3][99] )

[0.01771063 0.0172669  0.01818824 ... 0.0170528  0.0181793  0.01800956] [0.00548034 0.00514132 0.00520031 ... 0.00496647 0.00517431 0.00521462]


## B.1 - B.3 we only save point value (median) now for rolling-Y approach for analysis / plot

### B.1 get lifetime (alpha_CH4/N2O) and save to excel

In [29]:
df_allSSP = []
for n in range(len(ssp)): 
    # alpha_DF
    df = pd.DataFrame(columns=['SSP', 'MY', 'alpha_ch4', 'alpha_n2o'])
    rows = []
    # loop over MY, 
    for i in range(len(y_index)):
        alf_ch4, alf_n2o = alpha_ch4_l_allSSP[n][i] , alpha_n2o_l_allSSP[n][i]
        new_row = {'SSP': ssp[n], 'MY': 2000 + y_index[i], 'alpha_ch4': np.median(alf_ch4), 'alpha_n2o': np.median(alf_n2o)}   
        rows.append(new_row)  # Append the row to the list
    df = pd.concat([df, pd.DataFrame(rows)], ignore_index=True)
    df_allSSP.append(df)

In [30]:
ch4_n2o_alpha_perMY = pd.concat(df_allSSP)
ch4_n2o_alpha_perMY

,SSP,MY,alpha_ch4,alpha_n2o
0,ssp119,2025,11.199661,102.781299
1,ssp119,2026,11.276409,102.593138
2,ssp119,2027,11.344275,102.413259
3,ssp119,2028,11.400994,102.241544
4,ssp119,2029,11.450459,102.077878
...,...,...,...,...
95,ssp585,2120,10.185887,87.796694
96,ssp585,2121,10.135794,87.737297
97,ssp585,2122,10.084533,87.679097
98,ssp585,2123,10.032012,87.622077


In [31]:
if not os.path.exists("output/RollY"):
    os.makedirs("output/RollY")
ch4_n2o_alpha_perMY.to_excel( "output/RollY/ch4_n2o_alpha_perMY.xlsx" , index = False)

### B.2 get ERF [W m-2 ppm for CO2, ppb for CH4/N2O ]

In [32]:
# ERF_DF
df_erf_allSSP = []
for n in range(len(ssp)): 
    # alpha_DF
    df_erf = pd.DataFrame(columns=['SSP', 'MY', 'co2_erf_1ppm', 'ch4_erf_1ppb', 'n2o_erf_1ppb'])
    rows = []
    # loop over MY, 
    for i in range(len(y_index)):
        co2_erf, ch4_erf, n2o_erf = y0_co2_erf_l_allSSP[n][i], y0_ch4_erf_l_allSSP[n][i], y0_n2o_erf_l_allSSP[n][i]
        new_row = {'SSP': ssp[n], 'MY': 2000 + y_index[i], 
               'co2_erf_1ppm': np.median(co2_erf[i]), 
               'ch4_erf_1ppb': np.median(ch4_erf[i]), 
               'n2o_erf_1ppb': np.median(n2o_erf[i])
              }   
        rows.append(new_row) 

    df_erf = pd.concat([df_erf, pd.DataFrame(rows)], ignore_index=True)
    df_erf_allSSP.append(df_erf)

In [33]:
majorghg_erf_perMY = pd.concat(df_erf_allSSP)
majorghg_erf_perMY

,SSP,MY,co2_erf_1ppm,ch4_erf_1ppb,n2o_erf_1ppb
0,ssp119,2025,0.015512,0.000438,0.003401
1,ssp119,2026,0.015121,0.000374,0.003514
2,ssp119,2027,0.015801,0.000347,0.002494
3,ssp119,2028,0.014798,0.000479,0.002999
4,ssp119,2029,0.014907,0.000426,0.003396
...,...,...,...,...,...
95,ssp585,2120,0.005552,0.000424,0.002650
96,ssp585,2121,0.004927,0.000352,0.002749
97,ssp585,2122,0.005098,0.000386,0.003034
98,ssp585,2123,0.004623,0.000359,0.002469


In [34]:
majorghg_erf_perMY.to_excel( "output/RollY/majorghg_erf_perMY.xlsx" , index = False)

### B.3 get gas concentration

In [35]:
#concentration_DF
df_C_allSSP = []
for n in range(len(ssp)): 
    # alpha_DF
    df_c = pd.DataFrame(columns=['SSP', 'MY', 'co2_concentration', 'ch4_concentration', 'n2o_concentration'])
    rows = []
    # loop over MY, 
    for i in range(len(y_index)):
        co2_c, ch4_c, n2o_c = y0_co2_C_l_allSSP[n][i], y0_ch4_C_l_allSSP[n][i], y0_n2o_C_l_allSSP[n][i]
        new_row = {'SSP': ssp[n], 'MY': 2000 + y_index[i], 
               'co2_concentration': np.median(co2_c[i]), 
               'ch4_concentration': np.median(ch4_c[i]), 
               'n2o_concentration': np.median(n2o_c[i])
              }   
        rows.append(new_row) 

    df_c = pd.concat([df_c, pd.DataFrame(rows)], ignore_index=True)
    df_C_allSSP.append(df_c)

In [36]:
majorghg_C_perMY = pd.concat(df_C_allSSP)
majorghg_C_perMY

,SSP,MY,co2_concentration,ch4_concentration,n2o_concentration
0,ssp119,2025,278.100333,729.200000,270.100000
1,ssp119,2026,281.364285,825.736408,271.314303
2,ssp119,2027,284.004997,912.714075,272.532667
3,ssp119,2028,286.506135,994.292614,273.774199
4,ssp119,2029,288.918566,1069.389034,275.018022
...,...,...,...,...,...
95,ssp585,2120,852.373135,2573.756524,367.858505
96,ssp585,2121,863.448637,2561.542353,368.418020
97,ssp585,2122,874.469148,2549.220472,368.963758
98,ssp585,2123,885.460453,2536.811917,369.495887


In [37]:
majorghg_C_perMY.to_excel( "output/RollY/majorghg_C_perMY.xlsx" , index = False)

# C. ModuleB: Calc and save RF (CRF / GWP saved ,but can't be used directly, need calculate from the new RF) 

### the new rolling_y [per Y] approach, we need y_index = [20,21,22,...], "H_max=0" for each Year, we kept H_max=10  though to explore patterns

In [38]:
ssp = ["ssp119", "ssp126", "ssp245",  "ssp585"]  #"ssp370", "ssp434", "ssp460",
""" starting year is 2000, then y_index for 2020, 2030... is 20 , 30 """
y_start = 2000 
H_m = 1
#y_index , each year defined above

for x in ssp: 
    for y in y_index: 
        f36 = majorghg_get_f(f = f, H_max=H_m, ts_per_year = 1, year_index = y , scn = x)  # 270 for year 2020 if starting 1750, ... 290 for year 2040

        n36 = majorghg_analysis(f = f, H_max=H_m, fair_start_y = y_start, ts_per_year = 1, year_index = y, scn = x)     
        
        nco2 = n36.co2_analytical2023(co2_erf_diff_t = f36.get_co2_1ppm_erf() )
        nch4 = n36.ch4_analytical2023(alpha_ch4 = f36.call_f_from_fair_gas()[1]["CH4_lifetime"], #index[0,1,2] -> CO2/CH4/N2O
                                      ch4_erf_diff_t = f36.get_ch4_1ppb_erf())
        nn2o = n36.n2o_analytical2023(alpha_n2o = f36.call_f_from_fair_gas()[2]["N2O_lifetime"],  
                                      ch4_erf_diff_t = f36.get_ch4_1ppb_erf(),
                                      n2o_erf_diff_t  = f36.get_n2o_1ppb_erf() )

        nch4_cc = n36.carbon_cycle_adjustment(agtp = nch4[2], rf_co2 = nco2[0], agwp_co2 = nco2[1], agtp_co2 = nco2[2], 
                                        iagtp_co2 = nco2[3]) 

        nn2o_cc = n36.carbon_cycle_adjustment(agtp = nn2o[2], rf_co2 = nco2[0], agwp_co2 = nco2[1], agtp_co2 = nco2[2], 
                                        iagtp_co2 = nco2[3]) 

        fch41, fch42, fch43 = n36.get_final_rf_agwp_gwp_ch4_n2o(rf_gas = nch4[0], rf_cc_gas=nch4_cc[0], agwp_gas = nch4[1], agwp_cc_gas = nch4_cc[1], agwp_co2 = nco2[1] )
        fn2o1, fn2o2, fn2o3 = n36.get_final_rf_agwp_gwp_ch4_n2o(rf_gas = nn2o[0], rf_cc_gas=nn2o_cc[0], agwp_gas = nn2o[1],  agwp_cc_gas = nn2o_cc[1] , agwp_co2 = nco2[1])
        fco21, fco22, fco23 = n36.final_rf_agwp_gwp_co2(rf_co2 = nco2[0], agwp_co2 = nco2[1])

        # finalgas_agwp_dcf_gwp: total 6 output: final_agwp, final_agwp_single, final_dcf, final_dcf_single, final_gwp, final_gwp_single
        finalch4_agwp_dcf_gwp = n36.get_dcf_finaloutput ("CH4", fch43, fch41, fch42 ) #rf, agwp, gwp
        finaln2o_agwp_dcf_gwp = n36.get_dcf_finaloutput ("N2O", fn2o3, fn2o1, fn2o2  )
        finalco2_agwp_dcf_gwp = n36.get_dcf_finaloutput ("CO2", fco23, fco21, fco22 )

    

analyzing for year: 2025
original AGWP and GWP dims is:  (2, 1001) (2, 1001) Hmax and ts per year is: 1 1
final dims is:  (2, 1001) (2, 1001) (2, 1001)
check dimentions: True
calculated metric saved to output/metrics_2024-06-28/agwp_dcf_gwp1_tstep1CH4_ssp119_fair_start2000MY2025.xlsx
original AGWP and GWP dims is:  (2, 1001) (2, 1001) Hmax and ts per year is: 1 1
final dims is:  (2, 1001) (2, 1001) (2, 1001)
check dimentions: True
calculated metric saved to output/metrics_2024-06-28/agwp_dcf_gwp1_tstep1N2O_ssp119_fair_start2000MY2025.xlsx
original AGWP and GWP dims is:  (2, 1001) (2, 1001) Hmax and ts per year is: 1 1
final dims is:  (2, 1001) (2, 1001) (2, 1001)
check dimentions: True
calculated metric saved to output/metrics_2024-06-28/agwp_dcf_gwp1_tstep1CO2_ssp119_fair_start2000MY2025.xlsx
analyzing for year: 2026
original AGWP and GWP dims is:  (2, 1001) (2, 1001) Hmax and ts per year is: 1 1
final dims is:  (2, 1001) (2, 1001) (2, 1001)
check dimentions: True
calculated metric sa